<a href="https://colab.research.google.com/github/aknip/Local_LLMs/blob/main/Zephyr_7B_Alpha_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick start to run Zephyr 7B Alpha on Google Colab

- Source: https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha/blob/main/colab-demo.ipynb
- Info: https://levelup.gitconnected.com/zephyr-7b-%CE%B1-a-low-cost-llm-can-perform-better-than-llama-70b-a2055dd028cb

other:
- https://github.com/aigeek0x0/zephyr-7b-alpha-langchain-chatbot

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate
# or !pip install transformers accelerate

## Load dependencies

In [2]:
import torch
from transformers import pipeline

## Download model and load pipeline

takes approx. 3 mins

In [3]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Prepare inputs

In [4]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant". For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate.",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>



## Generate!

takes approx 40 secs on Colab T4

In [6]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Ahoy there me hearty! As for yer question, me believe that no human can eat a helicopter in one sitting, nor in multiple sittings, as it's not food! But if ye're hungry, me can suggest some hearty seafood dishes that'll make yer belly full and happy!
